<a href="https://colab.research.google.com/github/Deminalla/NLP/blob/main/fta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install scikit-learn

In [ ]:
# ekperimentas 1
# llama3 modelis geras lietuviu kalbai
# suskaiciuoti tikslumas, darant klasifikacija
# use prompts/instrukcija
https://www.kaggle.com/code/lucamassaron/fine-tune-llama-3-for-sentiment-analysis
# modelio pacio nekeiciam ir nemokinam, tsg jam duodam duomenis ir jis klasifikuoja kaip moka

# ekperimentas 2
# transfer learning arba fine-tuning
https://huggingface.co/docs/transformers/tasks/sequence_classification
# cia vyks mokymas

# ekperimentas 3
# gebetu identifikuot kur yra konkretus fta pastraipoje
# rodyti indeksus kur prasidejo ir baigesi, jeigu nfta tai 0, 0
# reiks keist exceli, kad rodyt pradzios intervala ir pabaigos
https://huggingface.co/docs/transformers/tasks/question_answering

# eksperimentas 4
# sugeneruot sakinių prieš/po pastraipą
# panaudot llama 3 multilingual for this
# reiktų palygint. Parrunint su ir be generated data, gale palygint
# hipotze: dirbtinai sugeneruotu duomenu pridejimas pagerina/pablogina fta pazinima
# ar tokio tipo uzdaviniams tai padeda ar ne

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

file_path = 'FTA_dataset.xlsx'
sheet1 = pd.read_excel(file_path, sheet_name='FTA', header=None)
sheet2 = pd.read_excel(file_path, sheet_name='NFTA', header=None)
sheet3 = pd.read_excel(file_path, sheet_name='index', header=None)

sheet1.columns = ['paragraph']
sheet2.columns = ['paragraph']
sheet3.columns = ['paragraph', 'start_index', 'end_index']

sheet1['fta'] = 1
sheet2['fta'] = 0

data = pd.concat([sheet1, sheet2], ignore_index=True)

# Shuffle the data
# sample() - randomly select data
# frac=1 - select 100% of data
# After shuffling, the original index of the DataFrame is no longer in order
# reset_index(drop=True) is used to reset the index to a new sequential order.
# drop=True - discards the old index
data = data.sample(frac=1).reset_index(drop=True)

# training (70%), validation (10%), and test (20%)
train, temp = train_test_split(data, test_size=0.3, random_state=42)
val, test = train_test_split(temp, test_size=(2/3), random_state=42)

train = train.reset_index(drop=True)
val = val.reset_index(drop=True)
test = test.reset_index(drop=True)

# Eksperimentas 1

In [ ]:
y_true = test["fta"].tolist()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("lightblue/suzume-llama-3-8B-multilingual")

In [ ]:
!pip install bitsandbytes
!pip install accelerate

In [ ]:
model = AutoModelForCausalLM.from_pretrained("lightblue/suzume-llama-3-8B-multilingual",
                                             device_map="auto",
                                             load_in_8bit=True)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "BSC-LT/salamandra-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             load_in_8bit=True,
                                             device_map="auto")

In [ ]:
def generate_prompt(data_point):
    return f"""
            Sociolingvistikos srityje Įvaizdžiui Grėsmingas Šnekos Aktas (ĮGŠA) yra komunikacinis veiksmas, kuris kelia grėsmę kalbėtojo ar klausytojo socialinei tapatybei ir savivertės jausmui.
            Pavyzdžiai ĮGŠA yra įžeidimai, pertraukimai ir kritika. Nekorektiško teksto pavyzdžiai:

            Aš stebiuosi, kad „Liaudies“ partija, taip skambiai pasivadinusi, ir nenori per dvi minutes suteikti laimės visai Lietuvos tautai.
            Gal tokia replika, čia kai kurie kandidatai, kalbėdami apie mokesčių surinkimą kitose savivaldybėse tiesiog gal nežino, kad Lietuvoje šiandien tiktai keturi miestai yra donorai: Vilnius, dvidešimt kartų mažesnis donoras Kaunas, Klaipėda ir Mažeikiai.
            Dabar dar vienas faktas. Jūsų Ministras, kurio pavardės nieks neatsimena, Gustas, sukūrė programą: „regio invest“, ar ne? Kuri turėjo padėti kilti regionams. 55 procentai tos programos buvo suinvestuota į Vilnių ir Kauną. Vėlgi ta pati ir ta pati tendencija, akivaizdžiai matosi, tiesiog yra didieji miestai, kurie yra preferuojami ir nieko regionams. Ir mes matom regionus, kur yra milžiniška emigraciją iš jų, milžiniška bedarbystė vis dar, 16 ir daugiau prcentų ir dar auganti, Ir dėl to nebuvo per keturis metus padaryta nei vieno žingsnio.

            Korektiško teksto pavyzdžiai:

            Dėl kolegos pasiūlymo dėl darbo vietų. Jeigu Darbo kodeksas bus toks, koks šiandien, tai jokių darbo vietų nebus, žmonės emigruos ir dar smarkiau. Apie tai smulkiau galėtų mūsų koalicijos partneris ir narys, Profsajungų centro pirmininkas Kęstutis Juknis.
            Na, gerai, tvarkoj. Turi, gerai, gerai. Gerai, gerai. Bet tada žinot, koks kyla klausimas? Kaip aš turiu elgtis. Kaip turi elgtis suaugęs žmogus, kada mato, kaip kitas suaugęs žmogus vilioja tokią nesupratingą mergaitę su saldainiais. Kaip jis turi elgtis? O aš matau šitoj studijoj tokį faktą. Ir aš nežinau, ką aš turiu daryt.
            Kad lietuvis grįžtų namo ir atsivežtų čia darbo rankas, reikia tam tikro susivokimo savyje.

            Išanalizuokite šį naujienų tekstą ir atsakykite, ar jame yra Įvaizdžiui Grėsmingas Šnekos Aktas.
            Atsakykite „1“ už „taip“ arba „0“ už „ne“.

            [{data_point["paragraph"]}] = """

In [ ]:
test1 = test
train1 = train
val1 = val

X_test = pd.DataFrame(test1.apply(generate_prompt, axis=1), columns=["paragraph"])
X_train = pd.DataFrame(train1.apply(generate_prompt, axis=1), columns=["paragraph"])
X_val = pd.DataFrame(val1.apply(generate_prompt, axis=1), columns=["paragraph"])

In [ ]:
from tqdm import tqdm

def predict(model, tokenizer):
    y_pred = []
    pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens = 1,
                do_sample=False
                )
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["paragraph"]
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        print("paragraph: ", test1.iloc[i]["paragraph"])
        print("answer: ", answer)
        if "1" in answer:
            y_pred.append(1)
        elif "0" in answer:
             y_pred.append(0)
        else:
            y_pred.append(-1)
    return y_pred

In [ ]:
y_pred = predict(model, tokenizer)

  1%|          | 1/143 [00:03<07:09,  3.03s/it]

paragraph:  Taip pat labai planuojama, bent jau deklaruojama, bet aš galvoju, kad vietos veiklos grupėse regionuose įgyvendins gerai savo strategiją ateinančiam laikotarpiui ir būtent Europos Sąjungos parama regionuose bus panaudota verslui, smulkiam vietų kūrimui. Be abejo, tai yra, reikia didint šeimos pajamas ir tai irgi per bendrą valstybės politiką, nes regionuose šeimos pajamos yra ypatingai svarbios, bet dideliam verslui, koncernui, ateinančiam, ir besikuriančiam regionuose, turėtų būt mokestinės lengvatos. Ir dar vienas dalykas, iš tikrųjų, reikia keisti mokestines sistemas, solidarumą, turėtų būt daugiau, kuomet turtingieji ne tik siekia pelno, bet didina atlyginimus ir darbuotojams. Ko nebus...darbo kodekse.
answer:   1


  1%|▏         | 2/143 [00:04<04:26,  1.89s/it]

paragraph:  Aš manau, kad kai kurie buvo užsidėję irgi iš šešėlinės ekonomikos paimt ten apie milijardą ar pusę milijardo tuo metu litų, bet taip jiems ir nepavyko, aš galiu aiškiai pasakyt ir tikriausiai tą visi žino, kad šita Vyriausybė kaip tik ir pavyko ir 2014-ais ir penkioliktais metais žymiai pagerinus mokesčių administravimą išimt iš šešėlinės ekonomikos dideles sumas pinigų. Nenoriu aš čia skaičių minėt, galėčiau įvardint, bet reikia pripažint, kad tai yra pripažinę, ką ir kalbėjo tarptautinės įvairiausios vertinimo institucijos, netgi profesoriaus pačio, paminėto mano Šnaiderio, eem, atlikta analizė ir pateikta Lietuvai. Aš manau, mums labai padėjo mokesčių administravimas, reiškia, įvykdyt, nacionalinio biudžeto pajamų surinkimą ir praeitais, ir užpraeitais metais ir įvertinus tai, kad mūsų prognozė dėl bendro vidaus produkto augimo nepasitvirtino dėl Rusijos, kaip sakyt, taikomo draudimo Lietuvos prekių eksportui, arba Rusijos ekonomikos nuosmukio, tai mes turim persiorient

  2%|▏         | 3/143 [00:05<03:27,  1.48s/it]

...

paragraph:  O Gabrielius Landsbergis jau Jūsų 120 paėmė kaip atskaitos tašką ir padidino dar dvigubai.
answer:   1


 99%|█████████▉| 142/143 [02:00<00:00,  1.35it/s]

paragraph:  Prie Jūsų padidėjo keturiais procentais 
answer:   1


100%|██████████| 143/143 [02:01<00:00,  1.18it/s]

paragraph:  Daugybė žmonių eilinių moka patį didžiausią pajamų mokestį, o daugybė kandidatuojančių milijonierių moka tris procentus, o vienas net ir nol tris procento. Tai aš grįžtu prie mūsų siūlymo, Vokietijos sistemoje tokių dalykų nėra, nes ten suskaičiuojamos absoliučiai visos pajamos, gautos iš darbo, iš miško, iš žemės ūkio, iš kitos veiklos, iš nuomos, iš butų nuomos, iš automobilių nuomos, viskas sudedama ir nuo to skaičiuojami mokesčiai. Lietuvoj gi daugybė privilegijuotų grupių. Tai mūsų koalica šitas privilegijuotas grupes ir privilegijuotas pensijas mažins, o didins mažiausias algas ir mažiausias pensijas
answer:   1


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate(y_true, y_pred):
    # Filter out undetermined predictions
    mask = [y != -1 for y in y_pred]
    filtered_y_true = [y_true[i] for i in range(len(y_true)) if mask[i]]
    filtered_y_pred = [y_pred[i] for i in range(len(y_pred)) if mask[i]]

    accuracy = accuracy_score(filtered_y_true, filtered_y_pred)
    precision = precision_score(filtered_y_true, filtered_y_pred, pos_label=1)
    recall = recall_score(filtered_y_true, filtered_y_pred, pos_label=1)
    f1 = f1_score(filtered_y_true, filtered_y_pred, pos_label=1)

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

In [ ]:
evaluate(y_true, y_pred)

Accuracy: 0.2727272727272727
Precision: 0.2642857142857143
Recall: 0.9736842105263158
F1 Score: 0.4157303370786517


# Eksperimentas 2

In [ ]:
id2label = {0: "NFTA", 1: "FTA"}
label2id = {"NFTA": 0, "FTA": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer

model2 = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-multilingual-uncased", num_labels=2, id2label=id2label, label2id=label2id)

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset

train2 = train.rename(columns={"fta": "label"})
val2 = val.rename(columns={"fta": "label"})
test2 = test.rename(columns={"fta": "label"})

train_dataset2 = Dataset.from_pandas(train2)
val_dataset2 = Dataset.from_pandas(val2)
test_dataset2 = Dataset.from_pandas(test2)

def preprocess_function(row):
    return tokenizer(row["paragraph"])

tokenized_train_data2 = train_dataset2.map(preprocess_function, batched=True)
tokenized_val_data2 = val_dataset2.map(preprocess_function, batched=True)

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
pip install evaluate

In [ ]:
import evaluate
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')

    accuracy_metric = accuracy.compute(predictions=predictions, references=labels)
    return {
        'accuracy': accuracy_metric['accuracy'],
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
import os
os.environ["WANDB_MODE"] = "disabled"

training_args = TrainingArguments(
    output_dir="model_2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to=None,
)

trainer = Trainer(
    model=model2,
    args=training_args,
    train_dataset=tokenized_train_data2,
    eval_dataset=tokenized_val_data2,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.427351,0.756757,0.000000,0.000000,0.000000
2,No log,0.358474,0.797297,1.000000,0.166667,0.285714
3,No log,0.317599,0.878378,1.000000,0.500000,0.666667
4,No log,0.294249,0.891892,0.812500,0.722222,0.764706
5,No log,0.281974,0.851351,0.666667,0.777778,0.717949


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=165, training_loss=0.4415164369525331, metrics={'train_runtime': 373.8805, 'train_samples_per_second': 6.954, 'train_steps_per_second': 0.441, 'total_flos': 394099249826880.0, 'train_loss': 0.4415164369525331, 'epoch': 5.0})

In [ ]:
trainer.save_model("model_2")

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="model_2", tokenizer=tokenizer)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

true_labels = []
predicted_labels = []

for data in test_dataset2:
    text = data["paragraph"]
    true_label = data["label"]

    result = classifier(text)
    predicted_label = result[0]["label"]

    # Map the true label to the corresponding label ("NFTA" or "FTA")
    if true_label == 0:
        true_label_mapped = "NFTA"
    else:
        true_label_mapped = "FTA"

    true_labels.append(true_label_mapped)
    predicted_labels.append(predicted_label)

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, pos_label="FTA")
recall = recall_score(true_labels, predicted_labels, pos_label="FTA")

metrics = {
    "accuracy": accuracy,
    "precision": precision,
    "recall": recall
}

print(metrics)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'accuracy': 0.8187919463087249, 'precision': 0.5434782608695652, 'recall': 0.8064516129032258}


In [ ]:
text = test_dataset2[0]
print(text["paragraph"])
print("label", text["label"])
classifier(text["paragraph"])

Šilumos kaina, niekam ne paslaptis, kad priklauso nuo energetinių išteklių kainos. Aš esu Energetikos komisijos narys ir seniau buvo pristatyta išvada apie dujų terminalo projekto įgyvendinimą Lietuvoje. Buvo pripažinta, kad tas dujų terminalas yra per didelis, per brangus. Ir pasirašyta sutartis, kad mes turim pirkti iš Statoilo 500 milijonų kubinių metrų dujų, kurie tuo laiku buvo brangesnė 20 procentų nei rinkos kaina. Ir kaip jūs norit, jūs, eeem, neišvengiamai pakilo šilumos kaina
label 0


[{'label': 'NFTA', 'score': 0.905535101890564}]

#Eksperimentas 3

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer

model3 = AutoModelForQuestionAnswering.from_pretrained("bert-base-multilingual-uncased")
tokenizer3 = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

In [3]:
sheet3['text_length'] = sheet3['paragraph'].apply(len) # remove these after bert model is changed, check the other mode's max length
sheet3 = sheet3[sheet3['text_length'] <= 512]

In [ ]:
!pip install datasets

In [48]:
from datasets import Dataset

sheet3['question'] = "Kur šioje pastraipoje yra Įvaizdžiui Grėsmingas Šnekos Aktas?"
data3 = sheet3.sample(frac=1).reset_index(drop=True)

# training (70%), validation (10%), and test (20%)
train3, temp3 = train_test_split(data3, test_size=0.3, random_state=42)
val3, test3 = train_test_split(temp3, test_size=(2/3), random_state=42)

train3 = train3.reset_index(drop=True)
val3 = val3.reset_index(drop=True)
test3 = test3.reset_index(drop=True)

train_dataset3 = Dataset.from_pandas(train3)
val_dataset3 = Dataset.from_pandas(val3)
test_dataset3 = Dataset.from_pandas(test3)

In [ ]:
train_dataset3[0]

{'paragraph': 'viešumas ir atvirumas yra  svarbiausias instrumentas kovojant su korupcija. Visa informacija apie viešuosius pirkimus, už kiek pirko, iš ko pirko, turi būti prieinama visuomenei. Maža to, visuomenė galėtų pareikalauti, naudojantis internetu, atlikti nepriklausmą tyrimą vieno ar kito pirkimo, kurį atliktų valstybės kontrolė ir būtų paviešinta ataskaita viešai. Taigi žmonės gali ir sugebėtų kontroliuoti daugelį procesų. O ne sistema būtų kalta.',
 'start_index': 0,
 'end_index': 0,
 'text_length': 446,
 'question': 'Kur šioje pastraipoje yra Įvaizdžiui Grėsmingas Šnekos Aktas?'}

In [8]:
def preprocess_function(examples):
    inputs = tokenizer3(
        examples["question"],
        examples["paragraph"],
        return_offsets_mapping=True,
        padding="max_length",
        truncation=True,
        max_length=512, # change max length after model change
    )

    offset_mapping = inputs.pop("offset_mapping")
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        start_char = examples["start_index"][i]
        end_char = examples["end_index"][i]
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1: # 0 for the question and 1 for the context
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_train_data3 = train_dataset3.map(preprocess_function, batched=True)
tokenized_val_data3 = val_dataset3.map(preprocess_function, batched=True)

In [50]:
from transformers import DefaultDataCollator, TrainingArguments, Trainer

data_collator = DefaultDataCollator()

In [51]:
import os
os.environ["WANDB_MODE"] = "disabled"

training_args = TrainingArguments(
    output_dir="model_3",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=False,
    logging_strategy="epoch",
)

trainer = Trainer(
    model=model3,
    args=training_args,
    train_dataset=tokenized_train_data3,
    eval_dataset=tokenized_val_data3,
    processing_class=tokenizer3,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.050000,2.547161
2,0.054800,2.346123
3,0.038700,2.186013
4,0.010900,2.583845
5,0.007100,2.577104


TrainOutput(global_step=120, training_loss=0.032305747270584106, metrics={'train_runtime': 212.2768, 'train_samples_per_second': 8.903, 'train_steps_per_second': 0.565, 'total_flos': 493850870231040.0, 'train_loss': 0.032305747270584106, 'epoch': 5.0})

In [52]:
trainer.save_model("model_3")

In [21]:
def calculate_statistics(dataset):
    total_accuracy = 0
    total_recall = 0
    total_sensitivity = 0
    num_samples = len(dataset)

    for sample in dataset:
        actual_start = sample['start_index']
        actual_end = sample['end_index']
        paragraph_length = sample['text_length']

        prediction = question_answerer(question=sample['question'], context=sample['paragraph'])

        predicted_start = prediction['start']
        predicted_end = prediction['end']

        accuracy = 1 if (actual_start != 0 or actual_end != 0) == (predicted_start != 0 or predicted_end != 0) else 0
        total_accuracy += accuracy

        # Calculate area of overlap
        overlap_start = max(actual_start, predicted_start)
        overlap_end = min(actual_end, predicted_end)
        overlap_area = max(0, overlap_end - overlap_start)

        # Calculate recall
        actual_length = max(1, actual_end - actual_start)  # Avoid division by zero
        recall = overlap_area / actual_length
        total_recall += recall

        # Calculate sensitivity
        sensitivity = overlap_area / paragraph_length
        total_sensitivity += sensitivity

    # Compute averages
    avg_accuracy = total_accuracy / num_samples
    avg_recall = total_recall / num_samples
    avg_sensitivity = total_sensitivity / num_samples

    return {
        'accuracy': avg_accuracy,
        'recall': avg_recall,
        'sensitivity': avg_sensitivity
    }

In [53]:
from transformers import pipeline
question_answerer = pipeline("question-answering", model="model_3", tokenizer=tokenizer3)

calculate_statistics(test_dataset3)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'accuracy': 0.45871559633027525,
 'recall': 0.03768066961711981,
 'sensitivity': 0.03044896860719102}

In [ ]:
question_answerer(question=test_dataset3[13]["question"], context=test_dataset3[11]['paragraph'])

{'score': 0.3976801037788391,
 'start': 21,
 'end': 55,
 'answer': 'Aš net neabejoju, kad jis atsakys!'}